In [17]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

X=torch.rand(2,20)
#向前传播
net(X)

tensor([[ 0.1998,  0.0329,  0.1711,  0.0778, -0.0741,  0.1283, -0.2254, -0.1535,
         -0.0229,  0.1079],
        [ 0.1381,  0.1017,  0.1175,  0.0825, -0.0968,  0.1987, -0.1645, -0.2656,
          0.0554,  0.0902]], grad_fn=<AddmmBackward0>)

In [18]:
class MLP(nn.Module):
    #定义需要哪些类，需要哪些参数
    def  __init__(self):
        super().__init__()
        #super(MLP, self).__init__() 这个是更准确的调用
        self.hidden=nn.Linear(20,256)
        self.out=nn.Linear(256,10)
        
    def forward(self,X):
        return self.out(F.relu(self.hidden(X)))
        

In [25]:
net=MLP()

net(X)

tensor([[ 0.1554,  0.0014,  0.0489,  0.0759,  0.2934,  0.1249, -0.1015, -0.1926,
          0.0321, -0.1030],
        [ 0.0741, -0.0343,  0.0410,  0.0122,  0.1413, -0.0933, -0.1930, -0.0969,
         -0.0293, -0.2343]], grad_fn=<AddmmBackward0>)

In [29]:
#自定义一个序列函数
class MySequential(nn.Module):
    def __init__(self,*args):
        super(MySequential, self).__init__()
        for block in args:
            self._modules[block]=block
    
    def forward(self,X):
        for block in self._modules.values():
            X=block(X)
        return X

net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))

net(X)

tensor([[-0.0876,  0.1187, -0.1335,  0.0291, -0.1634,  0.0528, -0.0967,  0.1745,
          0.1040,  0.1310],
        [-0.2604,  0.1244, -0.1646,  0.0576, -0.0420,  0.0054, -0.1078,  0.0338,
          0.1768,  0.1164]], grad_fn=<AddmmBackward0>)

In [31]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super(FixedHiddenMLP,self).__init__()
        #下面不参与训练，不会计算梯度
        self.rand_weight=torch.rand((20,20),requires_grad=False)
        self.linear=nn.Linear(20,20)
    def forward(self,X):
        X = self.linear(X)
        #手写X与randweight矩阵乘法
        X = F.relu(torch.mm(X,self.rand_weight)+1)
        X = self.linear(X)
        #当X的绝对值之和大于1一直除一个数
        while(X.abs().sum()>1):
            X/=2
        return X.sum()

net=FixedHiddenMLP()
net(X)

tensor(0.2066, grad_fn=<SumBackward0>)

In [32]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.0830, grad_fn=<SumBackward0>)